# Hướng dẫn thực hành tuần 1 - Introduction to Data Science


## 0. Thêm thư viện

In [ ]:
import requests
import os
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import sqlite3

**Mô tả về các thư viện sử dụng**

Các bạn có thể truy cập vào link hướng dẫn của các thư viên

- **[requests](https://requests.readthedocs.io/en/latest/)**: Là thư viện hỗ trợ tương tác đến API để thực hiện thao tác cào dữ liệu
- **[os](https://docs.python.org/3/library/os.html)**: Là thư viện để tương tác, hoạt động với interfaces của hệ thống.
- **[pandas](https://pandas.pydata.org/docs/)**: Là thư viện sử dụng để đọc dữ liệu và xử lý dữ liệu liên quan đến công việc
- **[selenium](https://selenium-python.readthedocs.io/)**: Truy cập và tương tác với trang web thông qua HTML.
- **[sqlite3](https://docs.python.org/3/library/sqlite3.html)**: Khởi tạo và xử lý các vấn đề liên quan đến CSDL


## 1. Thu thập dữ liệu

### 1.1 Dữ liệu từ API

#### a. Truy cập dữ liệu

In [ ]:
url = 'https://api.publicapis.org/entries' #Link API

In [ ]:
request = requests.get(url) #Lấy thông tin từ API cần truy cập

In [ ]:
#Kiểm tra kết quả truy cập
if request.status_code == 200:
    print("Truy cập thành công")
else:
    print(request.status_code)

In [ ]:
request.text #Đọc dữ liệu dưới dạng text

In [ ]:
data = request.json() #Đọc dữ liệu dưới dạng json (Kiểu dict trong python)
print(data)

**Ví dụ về cách lưu trữ của kiểu dict**
![Example_Dict](./dict.png "Dictionary in Python")

In [ ]:
data.keys() # Kiểm tra các key tồn tại trong data

In [ ]:
data['count'] # Kiểm tra giá trị data vs key là count

In [ ]:
data['entries'] # Kiểm tra nội dung data vs key là entries

In [ ]:
type(data['entries']) # Kiểm tra kiểu dữ liệu của giá trị data['entries'] 

In [ ]:
len(data['entries']) # Kiểm tra kích thước dữ liệu của giá trị data['entries'] 

#### b. Lưu dữ liệu dưới dạng CSV

**Mục tiêu:** Chuyển dữ liệu từ một mảng với nhiều phần tử dict -> kiểu dữ liệu DataFrames -> Thực hiện lưu thành file dữ liệu "data.csv"

In [ ]:
df = pd.DataFrame(data['entries']) # Chuyển từ dữ liệu từ mảng dict sang DataFrames

In [ ]:
df

In [ ]:
df.to_csv('./data.csv',index=False) #Lưu xuống thành file "data.csv".

### 1.2 Tải dữ liệu trực tiếp thông qua HTML

**Mục tiêu:** Tìm bảng dữ liệu trên 1 trang bất kỳ. Đọc thông tin từ bảng và chuyển thành kiểu dữ liệu DataFrame

In [ ]:
url_fb = "https://www.tutorialrepublic.com/sql-tutorial/sql-dates-and-times.php"

driver = webdriver.Chrome(ChromeDriverManager().install()) # Mở trình duyệt

driver.get(url_fb) # Truy cập đường link

# driver.page_source lấy toàn bộ thông tin html của trang

data = pd.read_html(driver.page_source) # Lấy các bảng dữ liệu xuất hiện trong bảng và chuyển thành DataFrame

print(data[0]) #Hiển thị thông tin bảng thứ nhất

print('Driver Title:',driver.title)
print('Driver URL:',driver.current_url)

driver.implicitly_wait(10)
driver.close() # Kết thúc đóng trình duyệt


## 2. Đọc dữ liệu

### 2.1 File .csv

**[pd.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)** (filepath_or_buffer, sep,skipinitialspace)

- **filepath_or_buffer**: Đường dẫn
- **sep**: Dấu ngăn cách
- **skipinitialspace**: Bỏ khoảng trắng sau dấu ngăn cách

In [ ]:
df = pd.read_csv("./data.csv") # đọc dữ liệu

In [ ]:
df

In [ ]:
df.head(5) #hiển thị 5 dòng đầu tiên

In [ ]:
df.tail(5) #hiển thị 5 dòng đầu cuối

### 2.2 Cơ sở dữ liệu SQL

**Khởi tạo cơ sở dữ liệu**

In [ ]:

con = sqlite3.connect('DS_database') 
cur = con.cursor()

cur.execute('''CREATE TABLE IF NOT EXISTS KIMDUNG
          ([id] INTEGER PRIMARY KEY, 
           [name] TEXT, 
           [money] INTEGER)''')

cur.execute('''INSERT INTO KIMDUNG (id, name, money)
                VALUES (1,'Doc Co Cau Bai',25000),
                (2,'Bac Kieu Phong',20000),
                (3,'Nam Mo Dung',15000),
                (4,'Truong Vo Ky',15000),
                (5,'Vo Danh Than Tang',28000)''') 

con.commit()

**Tạo câu truy vấn**

In [ ]:
sql_query = 'SELECT * FROM KIMDUNG'

**Truy vấn đến cơ sở dữ liệu**



**[pd.read_sql()](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html)** (**sql**,**con**):
- **sql:** Câu lệnh truy vấn SQL hoặc tên bảng
- **con:** Cơ sở dữ liệu

In [ ]:
df = pd.read_sql(sql_query, con)

In [ ]:
df

**Ví dụ truy vấn: Lấy ra người có mức lương > 20000**

**[pd.read_sql_query()](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html) (sql,con):**
- **sql:** Câu lệnh truy vấn SQL
- **con:** Cơ sở dữ liệu

In [ ]:
sql_query = pd.read_sql_query('''SELECT * FROM KIMDUNG where money > 20000''', con)

In [ ]:
sql_query